In [5]:
import pandas as pd

In [6]:
data = pd.read_csv('../data/Cleaned_dataset.csv')

In [502]:
# Ingest the Data into the Knowledge Base

import minsearch

documents = data.to_dict(orient='records')

index = minsearch.Index(
    ['question', 'answer', 'category', 'difficulty_level', 'tags', 
     'example_query', 'explanation', 'common_mistakes', 'related_questions'],
    keyword_fields=['question', 'answer']
)

index.fit(documents)

query = "Give me intermediate SQL interview questions"
search_results = index.search(query, num_results=1)
print(search_results)


[{'Unnamed: 0': 516, 'question': 'What is a JOIN in SQL?', 'answer': 'A JOIN is used to combine rows from two or more tables based on a related column', 'category': 'SQL', 'difficulty_level': 'intermediate', 'tags': 'JOIN, SQL', 'example_query': 'SELECT * FROM employees INNER JOIN departments ON employees.department_id = departments.id;', 'explanation': 'JOINS are fundamental for retrieving related data across multiple tables', 'common_mistakes': 'Not understanding different types of JOINS (INNER, LEFT, RIGHT)', 'related_questions': 'What are CROSS JOINs?; Explain OUTER JOINS'}]


In [506]:
# 

In [507]:
# Configure OpenAI API for RAG
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role": "user", "content": query}]
)

response.choices[0].message.content


"Sure, here are some intermediate-level SQL interview questions that you might find useful:\n\n1. **Subqueries and Nested Queries:**\n   - What is a subquery in SQL and how is it different from a JOIN? Provide an example.\n   - How can you use a subquery to filter records in the outer query?\n\n2. **Joins:**\n   - Explain the differences between INNER JOIN, LEFT JOIN, RIGHT JOIN, and FULL OUTER JOIN.\n   - How would you return records that have matching entries in two tables along with unmatched records from one table?\n\n3. **Set Operations:**\n   - What are SQL set operations and when would you use UNION vs UNION ALL?\n   - How do INTERSECT and EXCEPT work in SQL?\n\n4. **Aggregations and Group By:**\n   - How does the GROUP BY clause work, and how would you use it to find the total revenue per customer?\n   - Can you filter groups in an SQL query? If so, how?\n\n5. **Indexes:**\n   - What are the different types of indexes in SQL and in what scenarios would you use each?\n   - How d

In [512]:
 # RAG flow implementation
def search(query):
    results = index.search(query, num_results=10)
    return results


prompt_template = """
You're a teaching assistant for SQL interview prep. Answer the QUESTION based on the CONTEXT from the database.
Use only the facts from the CONTEXT to answer the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

entry_template = """
question: {question}
answer: {answer}
category: {category}
difficulty_level: {difficulty_level}
tags: {tags}
example_query: {example_query}
explanation: {explanation}
common_mistakes: {common_mistakes}
related_questions: {related_questions}
""".strip()

# Build the prompt for the LLM based on the search results
def build_prompt(query, search_results):
    context = "\n\n".join([entry_template.format(**doc) for doc in search_results])
    prompt = prompt_template.format(question=query, context=context)
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',  
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# Putting it all together in the RAG function
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

query = "How do I create a CTE in SQL?"
print(rag(query))


To create a Common Table Expression (CTE) in SQL, you would use the `WITH` clause followed by a name for the CTE and the query that defines it. Here's the syntax:

```sql
WITH CTE_Name AS (
    -- Your query here
)
SELECT * FROM CTE_Name;
```

For example, a CTE can be created to find sales with amounts greater than 100:

```sql
WITH Sales_CTE AS (SELECT * FROM sales WHERE amount > 100)
SELECT * FROM Sales_CTE;
```

This structure allows you to reference the temporary result set produced by the CTE in a subsequent SELECT, INSERT, UPDATE, or DELETE statement, improving the readability and organization of complex queries.


In [514]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Assuming `data` is your DataFrame and `index` is your minsearch Index object

# Vectorize questions for vector-based search
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['question'])

def vector_search(query, num_results=10):
    query_vec = tfidf_vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    doc_indices = cosine_similarities.argsort()[-num_results:][::-1]
    return [documents[idx] for idx in doc_indices]

# Combining keyword and vector-based search (Hybrid Search)
def hybrid_search(query):
    keyword_results = index.search(query, num_results=5)  # Text-based search
    vector_results = vector_search(query, num_results=5)  # Vector-based search
    
    # Combine results
    combined_results = keyword_results + vector_results
    
    # Calculate cosine similarity for each result
    query_vec = tfidf_vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    
    # Create a list of tuples (result, similarity) for sorting
    result_with_similarity = [(result, sim) for result, sim in zip(combined_results, similarities)]
    
    # Sort by similarity score
    ranked_results = sorted(result_with_similarity, key=lambda x: x[1], reverse=True)
    
    # Return only the results without similarity scores
    return [result for result, _ in ranked_results[:10]]

# Implementing Query Rewriting
def rewrite_query(query):
    rewrite_prompt = f"Rewrite this query to be more effective for SQL interview questions retrieval: '{query}'"
    rewritten_query = llm(rewrite_prompt)  # Assuming llm is defined elsewhere
    return rewritten_query

# Updated RAG function with hybrid search and query rewriting
def advanced_rag(query):
    rewritten_query = rewrite_query(query)
    search_results = hybrid_search(rewritten_query)
    prompt = build_prompt(rewritten_query, search_results)  # Assuming build_prompt is defined elsewhere
    answer = llm(prompt)  # Assuming llm is defined elsewhere
    return answer

# Example usage
query = "How to join two tables in SQL?"
print(advanced_rag(query))

To prepare for SQL interviews specifically focused on joining two tables, you can refer to the following facts distilled from the provided context:

1. **Inner Join**: 
   - **Definition**: An inner join returns records that have matching values in both tables.
   - **Example Query**: 
     ```sql
     SELECT * FROM employees INNER JOIN departments ON employees.department_id = departments.id;
     ```
   - **Common Mistake**: Not understanding how inner joins filter out non-matching records.

2. **Outer Join**: 
   - **Definition**: An outer join returns all records from one table and the matched records from the other.
   - **Example Query**: 
     ```sql
     SELECT * FROM employees LEFT JOIN departments ON employees.department_id = departments.id;
     ```
   - **Common Mistake**: Misusing the type of outer join can lead to missing data.

3. **Self-Join**: 
   - **Definition**: A self-join is a regular join that joins a table to itself to combine rows based on related columns.
   - 

In [519]:
from sklearn.metrics import precision_score, recall_score
import numpy as np

def evaluate_retrieval(query, ground_truth, method="keyword"):
    # Get results based on the method
    if method == "keyword":
        search_results = index.search(query, num_results=10)
    elif method == "vector":
        # Assuming we implemented vector search earlier
        search_results = vector_search(query)

    # Retrieve indices from search results
    retrieved_ids = [data.index[data['question'] == doc['question']].tolist()[0] for doc in search_results]
    
    # Create binary arrays for precision and recall calculation
    y_true = np.zeros(len(data), dtype=int)
    y_pred = np.zeros(len(data), dtype=int)

    # Mark relevant IDs in y_true
    y_true[ground_truth] = 1  # Assuming ground_truth contains valid indices

    # Mark retrieved IDs in y_pred
    y_pred[retrieved_ids] = 1  # Assuming retrieved_ids contains valid indices

    # Precision and recall calculation
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    return {"precision": precision, "recall": recall}

# Example evaluation
query = "SQL intermediate question"
ground_truth = [1, 5, 7]  # Pretend these are relevant IDs
eval_results = evaluate_retrieval(query, ground_truth, method="keyword")
print("Keyword Search Results: ", eval_results)

# Compare with vector search
eval_results_vector = evaluate_retrieval(query, ground_truth, method="vector")
print("Vector Search Results: ", eval_results_vector)

Keyword Search Results:  {'precision': np.float64(0.0), 'recall': np.float64(0.0)}
Vector Search Results:  {'precision': np.float64(0.0), 'recall': np.float64(0.0)}


In [529]:
lengths = data['related_questions'].apply(lambda x: len(x) if isinstance(x, str) else 0)

print(data[lengths > lengths.mean()][['related_questions']])

                                     related_questions
0    Candidates might forget to use COALESCE, resul...
1    The query first creates a Common Table Express...
3    This query joins the Sales and Products tables...
5    This SQL query uses a JOIN to combine the 'emp...
9    This query first calculates total sales for ea...
..                                                 ...
495  Candidates might forget to include the proper ...
496  This query joins the 'products' and 'sales' ta...
497  This query joins the `sales` and `products` ta...
498  This query joins `customers` and `orders` to f...
499  Candidates might forget to apply the correct d...

[340 rows x 1 columns]
